In [1]:
import numpy as np
import open3d as o3d

from superprimitive_fusion.scanner import capture_spherical_scans
from superprimitive_fusion.utils import bake_uv_to_vertex_colours
from superprimitive_fusion.mesh_fusion_utils import get_mesh_components
from superprimitive_fusion.mesh_fusion import fuse_meshes

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
mesh = o3d.io.read_triangle_mesh("../data/power-drill/textured.obj", enable_post_processing=True)

bake_uv_to_vertex_colours(mesh)

mesh.compute_vertex_normals()

bb = mesh.get_minimal_oriented_bounding_box()
scale = np.mean(bb.get_max_bound())

In [3]:
scans = capture_spherical_scans(
    mesh=mesh,
    mask_generator=None,
    num_views=5,
    radius=0.3,
    width_px=360,
    height_px=240,
    fov=70,
    dropout_rate=0,
    depth_error_std=0.0,
    translation_error_std=0,
    rotation_error_std_degs=0,
    k=10,
    sampler="fibonacci",
)
meshes = [scan['mesh'] for scan in scans]

In [4]:
o3d.visualization.draw_geometries(
    meshes,
    window_name="Virtual scan",
    front=[0.3, 1, 0],
    lookat=[0, 0, 0],
    up=[0, 0, 1],
    zoom=0.7,
)

In [ ]:
j = 5
fused_mesh = meshes[0]
for i in range(1,j):
    fused_mesh = fuse_meshes(
        fused_mesh,
        meshes[i],
        h_alpha=3,
        nrm_shift_iters=2,
        shift_all=False,
        fill_holes=False,
    )

In [6]:
o3d.visualization.draw_geometries(
    [fused_mesh],
    window_name="Virtual scan",
    front=[0.3, 1, 0],
    lookat=[0, 0, 0],
    up=[0, 0, 1],
    zoom=0.7,
)

In [7]:
get_mesh_components(fused_mesh, show=False)

There are 2 clusters of connected components


[TriangleMesh with 39069 points and 77878 triangles.,
 TriangleMesh with 39069 points and 2 triangles.]